For more information about how to setup google sheets api:

Portuguese: https://developers.google.com/sheets/api/quickstart/python?hl=pt-br

English: https://developers.google.com/sheets/api/quickstart/python

``` python
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

In [3]:
import pandas as pd
from config import *
from gsheets_utils_requests import *

In [4]:
gs_client = GoogleSheetsClient(client_id=client_id,
                               client_secret=client_secret,
                               refresh_tkn=refresh_token)

Successfully connected!


# CONTROLE DE PROJETO

In [5]:
csc_data = gs_client.get_sheet_data(
    spreadsheet_id='1Q-P0QxPWxH_rCLslEFetiht9-SmmGtwyd5TPsm2HAp0',
    range_name='Tarefas!A1:T1000'
)

In [6]:
pmo_data = gs_client.get_sheet_data(
    spreadsheet_id='1aD6NWbiaIS1H4N09nkimKGeKJeD9ux2coW9aY2ZM-rc',
    range_name='Tarefas!A1:T1000'
)

In [7]:
tlog_data = gs_client.get_sheet_data(
    spreadsheet_id='128DTCwkR6DcthbwQzGmlGfAUnv6KuNidlE4_RWhfiXY',
    range_name='Tarefas!A1:T3000'
)

In [8]:
# engagement_survey_data = gs_client.get_sheet_data(
#     spreadsheet_id='1Rxo642UduYjf1OvHmvgXS5UN4OBwXK1nfOOWXMcYBrA',
#     range_name='Respostas ao formulário 1!A1:P100'
# )

# Cleaning Data

In [9]:
def clean_and_transform_data(data):
    # Extrair colunas e dados
    columns = data[0]  # Primeira linha são os nomes das colunas
    data = data[1:]    # As linhas subsequentes são os dados

    # Corrigir linhas com menos colunas
    fixed_data = [row + [''] * (len(columns) - len(row)) for row in data]

    # Substituir caracteres especiais e espaços nos nomes das colunas
    clean_columns = [col.replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_").replace("ç", "c").replace("ã", "a") for col in columns]

    # Criar o DataFrame do Pandas com os nomes de colunas limpos
    df = pd.DataFrame(fixed_data, columns=clean_columns)

    # Substituir strings vazias por None (valores nulos)
    df.replace('', None, inplace=True)
    
    # Converter colunas específicas para int, preenchendo valores inválidos com NaN
    df['Sprint'] = pd.to_numeric(df['Sprint'], errors='coerce')
    df['ID'] = pd.to_numeric(df['ID'], errors='coerce')

    df['Tempo_Efetivo_horas'] = df['Tempo_Efetivo_horas'].str.replace(',', '.').astype(float)
    df['Tempo_Estimado_horas'] = df['Tempo_Estimado_horas'].str.replace(',', '.').astype(float)

    # Converter colunas 'Inicio' e 'Conclusao' para datetime
    df['Inicio'] = pd.to_datetime(df['Inicio'], format='%d/%m/%Y', errors='coerce').dt.date
    df['Conclusao'] = pd.to_datetime(df['Conclusao'], format='%d/%m/%Y', errors='coerce').dt.date

    for column in df.columns:
        null_count = df[column].isnull().sum()
        print(f"Valores nulos em '{column}': {null_count}")
    
    return df

In [10]:
cleaned_csc_df = clean_and_transform_data(csc_data)

cleaned_csc_df.head()

Valores nulos em 'Sprint': 0
Valores nulos em 'ID': 444
Valores nulos em 'CTI': 437
Valores nulos em 'Classificacao': 826
Valores nulos em 'Task': 0
Valores nulos em 'Descricao': 646
Valores nulos em 'Entrega': 0
Valores nulos em 'Produto': 0
Valores nulos em 'Responsavel': 0
Valores nulos em 'Marcador': 826
Valores nulos em 'Funcao': 0
Valores nulos em 'Bitrix?': 0
Valores nulos em 'Status': 0
Valores nulos em 'Inicio': 0
Valores nulos em 'Conclusao': 0
Valores nulos em 'Observacao': 826
Valores nulos em 'Tempo_Estimado_horas': 0
Valores nulos em 'Tempo_Efetivo_horas': 0
Valores nulos em 'Tempo_Efetivo_Repasse_horas': 824
Valores nulos em 'Id_task_atrasada': 826


,Sprint,ID,CTI,Classificacao,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Funcao,Bitrix?,Status,Inicio,Conclusao,Observacao,Tempo_Estimado_horas,Tempo_Efetivo_horas,Tempo_Efetivo_Repasse_horas,Id_task_atrasada
0,1,80416.0,06.04.01-99,None,Edenred Serviços] Reuniões do sprint,None,Reuniões,Gestão,Viviane Pagnussat Cechetti,None,Data Analytics,Sim,Completed,2022-08-08,2022-08-19,None,12.0,13.33,None,None
1,1,80420.0,06.04.01-99,None,Edenred Serviços] Reuniões do sprint,None,Reuniões,Gestão,Ana Claudia Garcia,None,Produto,Sim,Completed,2022-08-08,2022-08-19,None,8.0,6.25,None,None
2,1,80422.0,06.04.03-01,None,Edenred Serviços] Gestão do projeto,None,Gestão do Projeto,Gestão,Bernardo Kuerten Dellagnelo,None,Projetos,Sim,Completed,2022-08-08,2022-08-19,None,12.0,13.50,None,None
3,1,80446.0,06.04.00-01,None,Edenred Serviços] Entendimento do ambiente Ed...,None,Entendimento dos dados,MIgração,Viviane Pagnussat Cechetti,None,Data Analytics,Sim,Completed,2022-08-08,2022-08-19,None,20.0,5.81,None,None
4,1,NaN,06.04.03-01,None,Ajuste de horas,None,Gestão do Projeto,Gestão,Ana Claudia Garcia,None,Produto,Sim,Completed,2022-08-08,2022-08-19,None,0.0,10.00,None,None


In [11]:
cleaned_tlog_df = clean_and_transform_data(tlog_data)

cleaned_tlog_df.head()

Valores nulos em 'Sprint': 5
Valores nulos em 'ID': 0
Valores nulos em 'CTI': 264
Valores nulos em 'Classificacao': 1076
Valores nulos em 'Task': 0
Valores nulos em 'Descricao': 892
Valores nulos em 'Entrega': 4
Valores nulos em 'Produto': 1
Valores nulos em 'Responsavel': 0
Valores nulos em 'Marcador': 1076
Valores nulos em 'Funcao': 0
Valores nulos em 'Bitrix?': 0
Valores nulos em 'Status': 0
Valores nulos em 'Inicio': 0
Valores nulos em 'Conclusao': 0
Valores nulos em 'Tempo_Estimado_horas': 2
Valores nulos em 'Tempo_Efetivo_horas': 0
Valores nulos em 'Tempo_Efetivo_Repasse_horas': 1076
Valores nulos em 'Id_task_atrasada': 1076
Valores nulos em 'Previsto_no_Planejamento?': 1076


,Sprint,ID,CTI,Classificacao,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Funcao,Bitrix?,Status,Inicio,Conclusao,Tempo_Estimado_horas,Tempo_Efetivo_horas,Tempo_Efetivo_Repasse_horas,Id_task_atrasada,Previsto_no_Planejamento?
0,1.0,71058,None,None,Suporte técnico,None,Supervisão Tecnica,QA,Vitor Paulon Avancini,None,Diretoria,Sim,Completed,2022-05-30,2022-06-10,20.0,0.00,None,None,None
1,1.0,70978,None,None,Gestão do projeto,None,Gestão do Projeto,Gestão,Bernardo Kuerten Dellagnelo,None,Projetos,Sim,Completed,2022-05-30,2022-06-10,20.0,22.75,None,None,None
2,1.0,70988,None,None,Acompanhamento do time e entendimento do negócio,None,Acompanhamento do time Edenred,QA,Vitor Gerber Weiss,None,Produto,Sim,Completed,2022-05-30,2022-06-10,64.0,27.00,None,None,None
3,1.0,70990,None,None,Acompanhamento do time e entendimento do negócio,None,Acompanhamento do time Edenred,QA,Carlos Alberto Juraszek Junior,None,Analytics Engineering,Sim,Completed,2022-05-30,2022-06-10,20.0,13.00,None,None,None
4,1.0,71060,None,None,Acompanhamento do time e entendimento do negócio,None,Acompanhamento do time Edenred,QA,Gabriel Lajús Maccarini,None,Analytics Engineering,Sim,Completed,2022-05-30,2022-06-10,48.0,8.96,None,None,None


In [12]:
cleaned_pmo_df = clean_and_transform_data(pmo_data)

cleaned_pmo_df.head()

Valores nulos em 'Sprint': 0
Valores nulos em 'ID': 0
Valores nulos em 'CTI': 2
Valores nulos em 'Classificacao': 213
Valores nulos em 'Task': 0
Valores nulos em 'Descricao': 125
Valores nulos em 'Entrega': 0
Valores nulos em 'Produto': 0
Valores nulos em 'Responsavel': 0
Valores nulos em 'Marcador': 213
Valores nulos em 'Funcao': 0
Valores nulos em 'Bitrix?': 0
Valores nulos em 'Status': 0
Valores nulos em 'Inicio': 0
Valores nulos em 'Conclusao': 0
Valores nulos em 'Tempo_Estimado_horas': 0
Valores nulos em 'Tempo_Efetivo_horas': 0
Valores nulos em 'Tempo_Efetivo_Repasse_horas': 212
Valores nulos em 'Id_task_atrasada': 211
Valores nulos em 'Previsto_no_Planejamento?': 213


,Sprint,ID,CTI,Classificacao,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Funcao,Bitrix?,Status,Inicio,Conclusao,Tempo_Estimado_horas,Tempo_Efetivo_horas,Tempo_Efetivo_Repasse_horas,Id_task_atrasada,Previsto_no_Planejamento?
0,1,215208,06.02.02-00,None,Desenvolvimento,None,Desenvolvimento,PMO BI,Camila Bosa Custódio,None,Analytics Engineering,Sim,Completed,2023-11-13,2023-11-24,40.0,16.0,None,None,None
1,1,215426,06.05.04-00,None,Supervisão Técnica,None,Supervisão Tecnica,Gestão,Viviane Pagnussat Cechetti,None,Data Analytics,Sim,Completed,2023-11-13,2023-11-24,10.0,0.0,None,None,None
2,1,215210,06.04.01-99,None,Gestão do projeto,None,Gestão do Projeto,Gestão,Higino Neto,None,Projetos,Sim,Completed,2023-11-13,2023-11-24,10.0,20.0,None,None,None
3,2,226118,06.04.01-99,None,Gestão do projeto,None,Gestão do Projeto,Gestão,Higino Neto,None,Projetos,Sim,Completed,2023-11-27,2023-12-08,10.0,0.0,None,None,None
4,2,226120,06.02.02-00,None,Desenvolvimento,None,Desenvolvimento,PMO BI,Camila Bosa Custódio,None,Analytics Engineering,Sim,Completed,2023-11-27,2023-12-08,40.0,18.0,None,None,None


# SNOWFLAKE

Snowflake-Snowpark-Python compatibility issue. Need to downgrade snowflake-snowpark-python to 1.9.0 (for Python 3.11, 3.10, 3.9, 3.8) and install snowflake connector using below command,( as per https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-pandas#installation)

´´´python
pip install snowflake-connector-python[pandas]
´´´

In [13]:
from config import *
from snowpark_utils import *
from snowflake.snowpark.functions import sproc

In [14]:
session = ConnectSession()

Starting connection...
Connected to Schema: "SANDBOX"."DEV_IVON_GARCIA"


## Create table on snowflake

1. Data Preprocessing:
* Fix Incomplete Rows: Corrects rows that have fewer columns than expected by padding them with empty strings.
* Clean Column Names: Replaces spaces and special characters in column names to ensure compatibility with Snowflake.

2. Creating the Pandas DataFrame: Uses the cleaned column names and fixed data.
    
3. Converting to Snowpark DataFrame: Converts the Pandas DataFrame to a Snowpark DataFrame.
    
4. Saving the DataFrame as a Table: Saves the DataFrame to Snowflake, using overwrite mode to replace any existing table with the same name.
    
5. Display Confirmation: Prints the first few rows of the newly created table to confirm successful creation.

In [15]:
def create_table(session, df, table_name):
    """
    Creates a table in Snowflake from a list of data. The first row of the data is used as column names.
    The table is created using Snowpark, and any existing table with the same name is overwritten.
    Use pip install snowflake-connector-python[pandas] to be able to create dataframe

    Parameters:
    session: Snowpark session object.
    data: List of lists containing the data. The first list should contain column names.
    table_name: The name of the table to be created in Snowflake.
    """

    if not isinstance(df, pd.DataFrame):
        raise ValueError("Data must be a pandas DataFrame")

    print("Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:")
    print(df.dtypes)

    # Convert the Pandas DataFrame to a Snowpark DataFrame
    snowpark_df = session.create_dataframe(df)
    
    # Insert the data into Snowflake and overwrite any existing table with the same name
    snowpark_df.write.mode("overwrite").save_as_table(table_name)

    # Print a message indicating the table was created and show the first few rows of the table
    print('Table created:')
    df_table = session.table(table_name)
    df_table.show()

In [16]:
create_table(session, cleaned_csc_df, table_name='teste_csc_2')
# create_table(session, cleaned_pmo_df, table_name='teste_pmo')
# create_table(session, cleaned_tlog_df, table_name='teste_tlog')

Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:
Sprint                           int64
ID                             float64
CTI                             object
Classificacao                   object
Task                            object
Descricao                       object
Entrega                         object
Produto                         object
Responsavel                     object
Marcador                        object
Funcao                          object
Bitrix?                         object
Status                          object
Inicio                          object
Conclusao                       object
Observacao                      object
Tempo_Estimado_horas           float64
Tempo_Efetivo_horas            float64
Tempo_Efetivo_Repasse_horas     object
Id_task_atrasada                object
dtype: object
Table created:
--------------------------------------------------------------------------------------------------------------------------

## Making stored procedure on snowflake

Instead of importing google sheets external utils, we will have to explicit write them here 

In [19]:
import pandas as pd
import requests
# from config import *

class GoogleSheetsClient:
    def __init__(self, client_id: str, client_secret: str, refresh_tkn: str):
        r = self.get_access_token(client_id, client_secret, refresh_tkn)
        data = r.json()
        self.token = data["access_token"]
        self.token_type = data["token_type"]

        print("Successfully connected!")


    def get_access_token(self, client_id: str, client_secret: str, refresh_tkn: str):
        url = "https://oauth2.googleapis.com/token"
        data = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_tkn,
            "client_id": client_id,
            "client_secret": client_secret
        }
        headers = {
            "Content-Type": "application/x-www-form-urlencoded"
        }

        r = requests.post(url, data=data, headers=headers)

        return r

    def connect_to_spreadsheet(self, spreadsheet_id: str, range_name: str):
        # Constructing the URL with the required parameters
        url = f"https://sheets.googleapis.com/v4/spreadsheets/{spreadsheet_id}/values:batchGet"
        params = {
        # Select your sheet and range you wish to ingest
            "ranges": range_name,
            "valueRenderOption": "FORMATTED_VALUE"
        }

        # Making the GET request to the Google Sheets API
        response = requests.get(url, headers={"Authorization": "Bearer " + self.token}, params=params)
        # Handling the response
        if response.status_code == 200:

            return response
        else:
            raise Exception(f"API Request Failed: {response.status_code} {response.text}")
    

    def get_sheet_data(self, spreadsheet_id, range_name):
        response = self.connect_to_spreadsheet(spreadsheet_id, range_name)
        json_data = response.json()
        values = json_data['valueRanges'][0]['values']

        # Convert the JSON data to a pandas DataFrame.
        # df = pd.DataFrame(values[1:], columns=values[0])

        return values

In [17]:
# session.add_packages("snowflake-snowpark-python", "pandas", "requests")
# session.sproc.register(func=create_table_from_gsheets, is_permanent=True, stage_location="@procedure",name="create_table_from_gsheets", replace=True)

@sproc(name="create_table_from_gsheets", is_permanent=True, stage_location="@procedure", replace=True, packages=["snowflake-snowpark-python", "pandas", "requests"])
def create_table_from_gsheets(session: Session, client_id: str, client_secret: str, refresh_token: str) -> int:
    gs_client = GoogleSheetsClient(client_id=client_id, client_secret=client_secret, refresh_tkn=refresh_token)
    data = gs_client.get_sheet_data(
        spreadsheet_id='1Q-P0QxPWxH_rCLslEFetiht9-SmmGtwyd5TPsm2HAp0',
        range_name='Tarefas!A1:T1000'
    )
    df = clean_and_transform_data(data)
    create_table(session, df, table_name='teste_csc_2')
    return 1  # Retornar um valor indicativo de sucesso

The version of package 'snowflake-snowpark-python' in the local environment is 1.18.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'pandas' in the local environment is 2.2.2, which does not fit the criteria for the requirement 'pandas'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'requests' in the local environment is 2.32.3, which does not fit the criteria for the requirement 'requests'. Your UDF might not work when the package version is different between the server and your local environment.


In [18]:
result = session.call("create_table_from_gsheets", client_id, client_secret, refresh_token)
result

SnowparkSQLException: (1304): 01b4ec43-0205-3e5a-0003-ecc7011349fa: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/usr/lib/python_udf/732626f7c31937c3233a79a8cef6bd8b3a427f1800cbfb15335faa2644d318b8/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python_udf/732626f7c31937c3233a79a8cef6bd8b3a427f1800cbfb15335faa2644d318b8/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python_udf/732626f7c31937c3233a79a8cef6bd8b3a427f1800cbfb15335faa2644d318b8/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^